In [1]:
# -*- coding: utf-8 -*-

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

import tensorflow as tf
import numpy as np
import time

def initialize_session():
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    return tf.Session(config=config)

/etc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
##################################################
BATCH_SIZE = 100
class_size = 4
use_clip = True         # Gradient clipping 쓸지 여부
train_keep_prob = 0.7
train_acc_summary = [] # 학습용 데이터의 정확도를 중간 중간 기록한다
test_acc_summary = []  # 테스트용 데이터의 정확도를 중간 중간 기록한다
train_loss_summary = [] # 학습용 데이터의 정확도를 중간 중간 기록한다
test_loss_summary = []  # 테스트용 데이터의 정확도를 중간 중간 기록한다
hypothesis_summary = []

max_len = 1000           # sequence 단어 수 제한
max_vocab = 50000       # maximum 단어 개수
emb_dim = 64         # 단어 embedding dimension
hidden_dim = 16        # RNN hidden dim
learning_rate = 0.0001  # Learning rate
use_clip = True         # Gradient clipping 쓸지 여부
##################################################

from genre_data_loader_merge_test import movie_data
data = movie_data(max_vocab=max_vocab, max_len=max_len)

train_total_batch = int(len(data.x_ids_train) / BATCH_SIZE)
test_total_batch = int(len(data.x_ids_test) / BATCH_SIZE)

Using TensorFlow backend.


In [3]:
print(len(data.x_ids_train))
print(len(data.y_train))

print(len(data.x_ids_test))
print(len(data.y_test))

print('train_total_batch',train_total_batch)
print('test_total_batch',test_total_batch)

5400
5400
600
600
train_total_batch 54
test_total_batch 6


In [4]:
from genre_movie_rnn_test import LSTM_Model
END_TOKEN = "<eos>"
lstm_model = LSTM_Model(max_len=max_len,
              emb_dim=emb_dim,
              hidden_dim=hidden_dim,
              vocab_size=max_vocab,
              class_size=class_size,
              use_clip=True, learning_rate=learning_rate, end_token=data.w2idx[END_TOKEN])


sess = initialize_session()
sess.run(tf.global_variables_initializer())

print(test_total_batch)

def test_model():
    test_loss, test_acc = 0, 0

    for _ in range(test_total_batch):
        test_x_txt, length, test_y = data.get_test(BATCH_SIZE)
        loss, acc = sess.run([lstm_model.loss, lstm_model.accuracy], feed_dict={lstm_model.x: test_x_txt, lstm_model.x_len: length, lstm_model.y: test_y, lstm_model.keep_prob: 1.0})
        
        #loss, acc = sess.run([model.loss, model.accuracy],
        #                      feed_dict={model.x_image: test_x, model.y_label: test_y, model.keep_prob : 1.0, model.is_training:False})
        #test_ids, length, label = data.get_test(BATCH_SIZE)
        #loss, acc = sess.run([model.loss, model.accuracy], feed_dict={model.x: test_ids, model.x_len: length, model.y: label})

        test_loss += loss
        test_acc += acc
        
    return test_loss/test_total_batch, test_acc/test_total_batch
    #Tensor("rnn/transpose_1:0", shape=(?, 25, 128), dtype=float32)

Tensor("Placeholder:0", shape=(?, 1000), dtype=int32)
Tensor("Placeholder_1:0", shape=(?,), dtype=int32)
Tensor("Placeholder_2:0", shape=(?,), dtype=int32)
6


In [5]:
for epoch in range(300):
    
    train_avg_loss, train_avg_acc = 0, 0
    
    for it in range(train_total_batch):
        train_x_txt, length, train_y = data.get_train(BATCH_SIZE)
        
        loss, acc, _, = sess.run([lstm_model.loss, lstm_model.accuracy, lstm_model.update],
                            feed_dict={lstm_model.x: train_x_txt, lstm_model.x_len: length, lstm_model.y: train_y, lstm_model.keep_prob: 0.8})
        
        train_avg_loss += loss / train_total_batch
        train_avg_acc += acc / train_total_batch
        
        #if it % 10 == 0 and it > 0:
            #model.save(sess)
            #print("* train_loss: {:.8f}, train_acc: {:.8f}".format(loss/BATCH_SIZE, acc))
            
        #if it % 20 == 0 and it > 0:
            #model.save(sess)
            #test_avg_loss, test_avg_acc = test_model()
            #print("** 100 test_loss: {:.8f}, test_acc: {:.8f}".format(test_avg_loss, test_avg_acc))
        
    test_avg_loss, test_avg_acc = test_model()
    
    train_acc_summary.append(train_avg_acc)
    test_acc_summary.append(test_avg_acc)
    train_loss_summary.append(train_avg_loss)
    test_loss_summary.append(test_avg_loss)
    #hypothesis_summary.append(hypothesis_)
    
    print("epoch {} - train_loss: {:.8f}, train_acc: {:.8f}, test_loss: {:.8f}, test_acc: {:.8f}"
          .format(epoch+1, train_avg_loss, train_avg_acc, test_avg_loss, test_avg_acc))  

epoch 1 - train_loss: 1.36833452, train_acc: 0.36481482, test_loss: 1.32078749, test_acc: 0.42000000
epoch 2 - train_loss: 1.18603264, train_acc: 0.49666666, test_loss: 1.13551774, test_acc: 0.48500000
epoch 3 - train_loss: 0.97521365, train_acc: 0.59925926, test_loss: 1.06327246, test_acc: 0.53166666
epoch 4 - train_loss: 0.79103233, train_acc: 0.70185186, test_loss: 0.99142344, test_acc: 0.57833332
epoch 5 - train_loss: 0.60373491, train_acc: 0.78462963, test_loss: 0.91898828, test_acc: 0.61166667
epoch 6 - train_loss: 0.43537151, train_acc: 0.84981481, test_loss: 0.89217926, test_acc: 0.63833334
epoch 7 - train_loss: 0.30263689, train_acc: 0.90629630, test_loss: 0.89000877, test_acc: 0.65833333
epoch 8 - train_loss: 0.20607025, train_acc: 0.94074074, test_loss: 0.90871930, test_acc: 0.65333334
epoch 9 - train_loss: 0.14012722, train_acc: 0.96555556, test_loss: 0.95534061, test_acc: 0.66000000
epoch 10 - train_loss: 0.09922389, train_acc: 0.97500001, test_loss: 0.98955929, test_acc: 

KeyboardInterrupt: 

In [ ]:
print('Max Train Accuracy {:.2f}%'.format(np.max(train_acc_summary)))
print('Max Test Accuracy {:.2f}%'.format(np.max(test_acc_summary)))

import matplotlib.pyplot as plt

# Get the figure and the axes
fig, (acc, loss) = plt.subplots(nrows=1,ncols=2, sharey=False, figsize=(15, 5))

# 첫번째 그래프
acc.plot(train_acc_summary, 'r', label='train')
acc.plot(test_acc_summary, 'b', label='test')
acc.legend(loc='upper left', frameon=False)
acc.set(title='Accuracy', xlabel='Epoch', ylabel='Accuracy')

# 두번째 그래프
loss.plot(train_loss_summary, 'r', label='train')
loss.plot(test_loss_summary, 'b', label='test')
loss.legend(loc='upper left', frameon=False)
loss.set(title='Loss', xlabel='Epoch', ylabel='Accuracy')

# Title the figure
fig.suptitle('TEST', fontsize=14, fontweight='bold')

In [ ]:
#model.restore(sess)

In [ ]:
#test_avg_loss, test_avg_acc = test_model()
#test_avg_acc

In [ ]:
hypothesis_summary[-1]

In [ ]:
w = hypothesis_summary[-1]
print(w.shape)
w = w.reshape(134,91,3,4)

w_min, w_max = np.min(w), np.max(w)

classes = ['Romance', 'Horror', 'Adventure', 'Documentary']
for i in range(4):
    plt.subplot(2,5,i+1)
    
    wimg=255.0*(w[:,:,:,i].squeeze() - w_min)/(w_max-w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])

In [ ]:
w = best_softmax.Weights[:-1, :]
w = w.reshape(32,32,3,10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2,5,i+1)
    
    wimg=255.0*(w[:,:,:,i].squeeze() - w_min)/(w_max-w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])

In [ ]:
tt = tf.nn.softmax()